In [1]:

from transformers import pipeline
sega = pipeline("text2text-generation",model='beyond/sega-large', device=1)

In [ ]:
'<mask> Conference on Empirical Methods <mask> submission of research papers <mask> Deep Learning <mask>'
'Bad news: <mask> the European Union <mask> month by EU <mask> Farm Commissioner Franz <mask>'

ss = ['nice weekend movie fun',
      'Shanghai food water help citizens virus',
      'England ship China ocean Paris cake people',
      'joint great food great drinks greater staff',
      'Wuhan hot-dry noodel delicious breakfirst China street Hubuxiang tour nice place',]

In [7]:
from transformers import pipeline
# sega = pipeline("text2text-generation",model='saved_models/bart-base-c4-realnewslike-4templates-passage-max15sents_2-sketch4/checkpoint-129375', framework='pt')
s = '<mask> machine learning <mask> my research interest <mask> data science <mask>'
sega(s, num_beams=3, do_sample=True, max_length=100)

[{'generated_text': 'I am interested in machine learning, and my research interest is in data science. I am a graduate student at the University of California, Davis.'}]

In [1]:
# sega-chinese
from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline
model_path = 'saved_models/bart-base-chinese-chinese_clean_passages_80m_with_sketch-10000000/checkpoint-93750'
tokenizer = BertTokenizer.from_pretrained(model_path)
sega_model = BartForConditionalGeneration.from_pretrained(model_path)
sega_generator = Text2TextGenerationPipeline(sega_model, tokenizer, device=7)

bart_model = BartForConditionalGeneration.from_pretrained('fnlp/bart-base-chinese')
bart_generator = Text2TextGenerationPipeline(bart_model, tokenizer, device=6)

2022-11-07 09:47:24.890192: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [6]:
bart_generator("今天[MASK]篮球[MASK]上海财经大学[MASK]", max_length=50, do_sample=False)

[{'generated_text': '今 天 的 篮 球 是 上 海 财 经 大 学 篮 球'}]

In [7]:
sega_generator("今天[MASK]篮球[MASK]上海财经大学[MASK]", max_length=50, do_sample=False)

[{'generated_text': '今 天 的 篮 球 比 赛 是 由 上 海 财 经 大 学 举 办 的 。'}]

In [3]:
sketchs = [
    # "今天[MASK]篮球[MASK]学校[MASK]",
    # "自然语言处理[MASK]谷歌[MASK]通用人工智能[MASK]",
    # "我昨天做了一个梦[MASK]又遇见了她[MASK]曾经那段时光让人怀恋[MASK]",
    "[MASK]疫情[MASK]公园[MASK]散步[MASK]",
    # "[MASK]酸菜鱼火锅[MASK]很美味，味道绝了[MASK]周末真开心[MASK]"
    ""
]
for sketch in sketchs:
    print('input sketch:\n>>> ', sketch)
    print('BART-chinese output:\n>>> ',bart_generator(sketch, max_length=100, do_sample=False)[0]['generated_text'].replace(' ',''))
    print('SEGA-chinese output:\n>>> ',sega_generator(sketch, max_length=100, do_sample=True, num_beams=3)[0]['generated_text'].replace(' ',''),'\n')


input sketch:
>>>  [MASK]疫情[MASK]公园[MASK]散步[MASK]
BART-chinese output:
>>>  在疫情防控公园内散步游玩
SEGA-chinese output:
>>>  为了防止疫情扩散，建议市民可以选择在公园内散步。 

input sketch:
>>>  
BART-chinese output:
>>>  
SEGA-chinese output:
>>>  若。 

